**Device config:**

In [ ]:
import getpass
hostname = input("Device Hostname:")
username = input("Username:")
password = getpass.getpass()

# CLI Commands

It is possible to run CLI commands on the Junos device using PyEZ as if you were using the CLI. However any output is text only and it is strongly recommended to use RPC commands which are discussed further below. In some rare cases there is no RPC command equivalent and it is necessary to use the CLI command.

Note: `dev.cli()` will not return any output automatically but will still run the commands. To see the output from the CLI command you muse use `print()`.


### CLI Command Example

In [ ]:
from jnpr.junos import Device

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

print(dev.cli("show route summary"))

dev.close()

# RPC Commands

Before you can run a RPC command you must know the RPC command name. You can use the CLI or PyEZ to get the RPC command name.

### Using the CLI to get the RPC command name

To get the RPC command name on the CLI use the "display xml rpc" command after any CLI command.

```
root@vsrx2> show route summary | display xml rpc 
<rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
    <rpc>
        <get-route-summary-information>
        </get-route-summary-information>
    </rpc>
    <cli>
        <banner></banner>
    </cli>
</rpc-reply>
```

### Using PyEZ to get the RPC command name

In [ ]:
from jnpr.junos import Device

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

print (dev.display_xml_rpc('show route', format='text'))

dev.close()

You only need to replace any hyphens (`-`) with underscores (`_`) to map it to the correct XML API RPC command name.

Example: `get-route-information` becomes `get_route_information`.

### Running RPC Commands

You can use the `rpc` property of any `Device` instance to run any RPC command available in the XML API. The `rpc` property will return an XML object starting at the first element under the <rpc-reply> tag.

In [ ]:
from jnpr.junos import Device
from lxml import etree

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

routes = dev.rpc.get_route_information()

dev.close()


### Printing RPC results

You cannot simply use `print()` to print the results of the RPC command:

In [ ]:
print(routes)

This is because the response is an XML tree

In [ ]:
type(routes)

In order to print the XML tree needs to be converted to a string using the `tostring()` method from `etree`.

In [ ]:
print(etree.tostring(routes, encoding='unicode'))

### Normalizing the output

You can normalize the output of the XML tree for all commands so that it is properly indented using `normalize=True` when the Device instance is created or when the `Device.open()` is used:

**Device Instantation Example:**

`dev = Device(host=hostname, user=username, password=password, port='22', normalize=True)`

**Device Open Example:**

`dev.open(normalize=True)`

**You can also normalize an individual RPC reply:**

`routes = dev.rpc.get_route_information(normalize=True)`



#### Normalizing Example

In [ ]:
from jnpr.junos import Device
from lxml import etree

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

routes = dev.rpc.get_route_information(normalize=True)
print(etree.tostring(routes, encoding='unicode'))

dev.close()

#### Why Normalizing is Important


xpath _______________________________

## JSON and Text RPC Output

You can also output the RPC reply in JSON or text formats. Text formats use a dictionary.
*Note: JSON is only supported on 14.2+*

In [ ]:
from jnpr.junos import Device
from lxml import etree

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

routes = dev.rpc.get_route_information({'format':'json'})
print(routes)

dev.close()